In [ ]:
import numpy as np
import torch
import clip
from clip.simple_tokenizer import SimpleTokenizer

from src import FoodDataModule, KPerClassSampler
from src import CLIP_Contrastive
from src import TextTransformer

from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from tqdm import tqdm

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
# Load the model
clip_backbone = "ViT-B/16"
device = "cuda" if torch.cuda.is_available() else "cpu"
model, image_transform = clip.load(clip_backbone, jit=False)
model = model.to(dtype=torch.float32)

input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

In [3]:
# Prepare dataset and loaders
dataset_root = "data/food-101/images"
datamodule = FoodDataModule(folder=dataset_root, 
                            batch_size = 100, 
                            image_transform=image_transform)
datamodule.setup()

In [4]:
def get_features(dataloader):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [5]:
ks = [1, 2, 4, 8, 16]

for k in ks:
    print(f'\nK: {k}')
    train_sampler = KPerClassSampler(dataset=datamodule.train_dataset, k=k, seed=42)
    
    train_loader = datamodule.train_dataloader(train_sampler, drop_last=False)
    test_loader = datamodule.test_dataloader(drop_last=False)
    
    # Calculate the image features
    train_features, train_labels = get_features(train_loader)
    test_features, test_labels = get_features(test_loader)

    # Perform logistic regression
    classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
    classifier.fit(train_features, train_labels)

    # Evaluate using the logistic regression classifier
    predictions = classifier.predict(test_features)
    accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
    print(f"Accuracy = {accuracy:.3f}")

  0%|                                                                                                                                                         | 0/1 [00:00<?, ?it/s]


K: 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [07:56<00:00,  2.36s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
/home/fabio/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0%|                                                                                                                                                         | 0/2 [00:00<?, ?it/s]

Accuracy = 50.936

K: 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [07:58<00:00,  2.37s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s finished
/home/fabio/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0%|                                                                                                                                                         | 0/4 [00:00<?, ?it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [07:58<00:00,  2.37s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s finished
/home/fabio/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0%|                                                                                                                                                        | 0/16 [00:00<?, ?it/s]

Accuracy = 79.490

K: 16


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 202/202 [07:56<00:00,  2.36s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 83.391


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.5s finished
/home/fabio/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
